<a href="https://colab.research.google.com/github/Richie1107/Image-Recognition/blob/main/%E6%A8%82%E9%80%8F%E9%97%9C%E8%81%AF%E6%80%A7%E9%A0%90%E6%B8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM


file_path = '/content/output.csv'  #使用爬蟲爬取樂透號碼，存成CSV檔

#讀取CSV檔
data = pd.read_csv(file_path)

#轉換成整數列表
def process_numbers(numbers_str):
    return [int(''.join(filter(str.isdigit, num))) for num in numbers_str.split(',')]

data['number'] = data['number'].apply(process_numbers)

#轉成二為數列
data_numbers = data['number'].tolist()

def to_binary_vector(groups, num_classes=49):
    binary_vectors = []
    for group in groups:
        vector = [0] * num_classes
        for number in group:
            vector[number - 1] = 1
        binary_vectors.append(vector)
    return np.array(binary_vectors)

binary_data = to_binary_vector(data_numbers)

#創造訓練集
time_steps = 1
X, y = [], []

for i in range(len(binary_data) - time_steps):
    X.append(binary_data[i:i + time_steps])
    y.append(binary_data[i + time_steps])

X = np.array(X)
y = np.array(y)

#重新制定格式
X = X.reshape((X.shape[0], X.shape[1], 49))
y = y.reshape((y.shape[0], 49))

#RNN模型
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(time_steps, 49)))
model.add(Dense(49, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy')

#訓練
model.fit(X, y, epochs=200, verbose=0)

#預測
y_pred = model.predict(X, verbose=0)

#轉化
def select_top_numbers(binary_vectors, num_predictions=6):
    top_numbers_list = []
    for vector in binary_vectors:
        top_indices = np.argsort(vector)[-num_predictions:]
        top_numbers = [index + 1 for index in top_indices]
        top_numbers_list.append(top_numbers)
    return top_numbers_list

def to_numbers(binary_vectors):
    selected_numbers = select_top_numbers(binary_vectors)
    return selected_numbers[0]  # 返回預測值最高的數組

predicted_numbers = to_numbers(y_pred)
print(predicted_numbers)


[[27, 36, 2, 32, 6, 43], [13, 20, 35, 30, 14, 22], [23, 34, 28, 11, 22, 39], [18, 9, 32, 36, 23, 37], [31, 40, 18, 48, 11, 23], [40, 15, 12, 34, 46, 30], [16, 34, 7, 32, 41, 43], [43, 19, 18, 4, 28, 23], [40, 8, 13, 2, 31, 15], [6, 32, 46, 41, 28, 34], [3, 2, 1, 43, 17, 33], [49, 28, 33, 8, 20, 13], [34, 14, 48, 33, 21, 20], [31, 25, 38, 22, 46, 49], [15, 17, 33, 39, 8, 5], [15, 38, 22, 9, 24, 3], [2, 17, 13, 29, 3, 26], [14, 48, 2, 35, 4, 8], [1, 37, 41, 49, 19, 12], [40, 2, 13, 30, 16, 43], [42, 2, 38, 29, 6, 46], [28, 16, 40, 48, 29, 41], [1, 8, 15, 14, 37, 12], [12, 5, 39, 8, 41, 18], [49, 45, 13, 22, 24, 16], [28, 40, 30, 38, 13, 7], [26, 44, 48, 15, 38, 31], [36, 26, 49, 2, 21, 15], [27, 29, 15, 39, 36, 2], [8, 3, 2, 9, 28, 20], [11, 7, 19, 4, 21, 33], [46, 25, 7, 1, 13, 35], [1, 7, 36, 46, 27, 42], [28, 38, 39, 3, 22, 17], [49, 31, 43, 2, 24, 6], [5, 26, 27, 48, 22, 49], [49, 1, 45, 33, 42, 14], [1, 23, 19, 17, 40, 33], [37, 44, 8, 11, 46, 36], [49, 2, 3, 47, 48, 27], [40, 10, 4

[27, 36, 2, 32, 6, 43]
